In [ ]:
# python libraries
import os
import numpy as np
import pandas as pd
import pyodbc
import glob
from datetime import datetime
from datetime import date
from dotenv import load_dotenv

In [ ]:
# drivers cherck
pyodbc.drivers()

In [ ]:
# .env
load_dotenv()

# info
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
server = os.getenv("DB_SERVER")
dbname = os.getenv("DB_NAME")

In [ ]:
# sql connection
conn_str = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={dbname};"
    f"UID={user};"
    f"PWD={password}"
)

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()


## Data to table SQL `final_result_tester`

In [ ]:
# file root
folder_path = r"\\myserver\\c1\\2025"

def get_all_csv_files():
    return glob.glob(os.path.join(folder_path, "*.csv"))

check_sql = "SELECT COUNT(1) FROM final_result_tester WHERE idrep = ?"

insert_sql = '''INSERT INTO final_result_tester (
    [date], [time], [model], [serial_number], [hipot_voltage],
    [hipot_leakage_current], [hipot_evaluation], [ground_bond_current],
    [ground_bond_voltage], [ground_bond_resistance], [ground_bond_evaluation],
    [free_run_voltage], [free_run_current], [free_run_power], [free_run_speed],
    [free_run_direction], [free_run_evaluation], [product_evaluation], [idrep]
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''

all_files = get_all_csv_files()

for file in all_files:
    df = pd.read_csv(file)
    df = df.replace({np.nan: None})
    df['idrep'] = df['Date'].astype(str) + df['Time'].astype(str)
    df['idrep'] = df['idrep'].str.replace(r"[^A-Za-z0-9]", "", regex=True)
    df['idrep'] = df['idrep'].str.replace(r"(AM|PM)$", r" \1", regex=True)

    for col in df.select_dtypes(include=['float']):
        df[col] = df[col].round(4)

    for _, row in df.iterrows():
        if not cursor.execute(check_sql, (row['idrep'],)).fetchone()[0]:
            try:
                cursor.execute(insert_sql, tuple(row))
            except Exception as e:
                print(f"Error al insertar fila: {e}")

conn.commit()


In [ ]:
cursor.close()
conn.close()
print("End.")